In [2]:
import json

with open("sq-api_table_paavo_pxt_12f1.px.json", "r", encoding="utf-8") as f:
    query_json = json.load(f)

url = "https://pxdata.stat.fi/PxWeb/api/v1/fi/Postinumeroalueittainen_avoin_tieto/uusin/paavo_pxt_12f1.px"


In [7]:
import requests

response = requests.post(url, json=query_json)
response.raise_for_status()
data = response.json()

# Show a summary of the returned keys and first items
print({k: (v[:1] if isinstance(v, list) else type(v)) for k, v in data.items()})


{'version': <class 'str'>, 'class': <class 'str'>, 'label': <class 'str'>, 'source': <class 'str'>, 'updated': <class 'str'>, 'note': ['... tieto on salassapitosäännön alainen'], 'role': <class 'dict'>, 'id': ['Postinumeroalue'], 'size': [3019], 'dimension': <class 'dict'>, 'extension': <class 'dict'>, 'value': [4290980]}


In [ ]:
import requests
import json

with open("sq-api_table_paavo_pxt_12f1.px.json", "r", encoding="utf-8") as f:
    query_json = json.load(f)

url = "https://pxdata.stat.fi/PxWeb/api/v1/fi/Postinumeroalueittainen_avoin_tieto/uusin/paavo_pxt_12f1.px"

response = requests.post(url, json=query_json)
response.raise_for_status()
data = response.json()


import pandas as pd
idx = data['dimension']['Postinumeroalue']['category']['label']
cols = data['dimension']['Vuosi']['category']['index']
# Values into matrix, row first
import numpy as np
vals = data['value']
vals = np.array(vals).reshape(len(idx),len(cols))


df = pd.DataFrame(index=idx.keys(),columns=cols.keys(),data=vals)




df['postal_area'] = idx.values()

In [39]:
df

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,postal_area
SSS,4290980,4319501,4347944,4374590,4396261,4414248,4431392,4446869,4459828,4476235,4492267,4512724,4537778,4581646,4622703,KOKO MAA
00100,14875,15204,15521,15529,15587,15830,15935,16159,16273,16163,16236,15817,15997,16365,16397,00100 Helsinki keskusta - Etu-Töölö (Helsinki)
00120,5816,5876,5904,5995,6031,6105,6116,6170,6202,6159,6221,6214,6209,6219,6317,00120 Punavuori - Bulevardi (Helsinki)
00130,1304,1316,1307,1274,1294,1340,1322,1307,1319,1374,1501,1480,1484,1467,1496,00130 Kaartinkaupunki (Helsinki)
00140,6636,6665,6713,6689,6712,6780,6866,6767,6800,6808,6872,6740,6674,6590,6712,00140 Kaivopuisto - Ullanlinna (Helsinki)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99930,197,205,204,204,205,201,205,212,205,199,196,199,195,195,195,99930 Sevettijärvi - Näätämö (Inari)
99940,10,16,13,15,13,15,15,14,16,22,23,21,21,22,26,99940 Näätämö (Inari)
99950,248,255,253,248,252,256,245,241,242,231,236,233,237,230,229,99950 Karigasniemi (Utsjoki)
99980,668,651,636,626,603,603,590,577,580,572,584,581,570,551,552,99980 Utsjoki Keskus (Utsjoki)


In [ ]:
import pandas as pd
import numpy as np

# Get years in order (sorted by index)
year_index = data['dimension']['Vuosi']['category']['index']
years = sorted(year_index.keys(), key=lambda x: year_index[x])
num_years = len(years)

# Get postal code areas in order
postal_index = data['dimension']['Postinumeroalue']['category']['index']
postal_labels = data['dimension']['Postinumeroalue']['category']['label']
postal_codes = sorted(postal_index.keys(), key=lambda x: postal_index[x])
num_postal_codes = len(postal_codes)

# Get values array
values = data['value']

# Verify the structure
print(f"Number of years: {num_years}")
print(f"Number of postal code areas: {num_postal_codes}")
print(f"Total values: {len(values)}")
print(f"Expected values: {num_years * num_postal_codes}")
print(f"Match: {len(values) == num_years * num_postal_codes}")

# Create DataFrame
rows = []
for i, postal_code in enumerate(postal_codes):
    start_idx = i * num_years
    end_idx = start_idx + num_years
    postal_values = values[start_idx:end_idx]
    
    for j, year in enumerate(years):
        rows.append({
            'Postinumeroalue': postal_code,
            'Postinumeroalue_label': postal_labels[postal_code],
            'Vuosi': year,
            'Value': postal_values[j]
        })

df = pd.DataFrame(rows)

print(f"\nDataFrame shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head(20))